## 建立電腦環境變數

In [ ]:
#建立電腦環境變數(模擬的)，並載入模擬環境
from dotenv import load_dotenv
load_dotenv()

## 連線資料庫

In [3]:
import psycopg2
import os
conn = psycopg2.connect(os.environ['postgre_password'])
conn.commit()
conn.close()

In [4]:
#使用 with as 語法可以自動 commit(), close() conn
with psycopg2.connect(os.environ['postgre_password']) as conn:
    cusor = conn.cursor()
    sql = '''
    CREATE TABLE IF NOT EXISTS 客戶(
	客戶_id SERIAL,
	客戶名稱 VARCHAR(255) NOT NULL,
	PRIMARY KEY(客戶_id)
    );
'''
    cusor.execute(sql)

    sql = '''
    CREATE TABLE IF NOT EXISTS 聯絡人(
	聯絡人id SERIAL,
	客戶id INT,
	聯絡人姓名 VARCHAR(255) NOT NULL,
	電話 VARCHAR(15),
	郵件 VARCHAR(100),
	PRIMARY KEY(聯絡人id),
	FOREIGN KEY(客戶id) REFERENCES 客戶(客戶_id)
    );
'''
    cusor.execute(sql)
    cusor.close()

In [ ]:
with psycopg2.connect(os.environ['postgre_password']) as conn:
    cusor = conn.cursor()
    sql = '''
    INSERT INTO 客戶(客戶名稱)
    VALUES('遠傳電信'),('台灣大車隊');
'''
    cusor.execute(sql)


    cusor.execute(sql)
    cusor.close()